# Regularization and Cross-Validation
*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. Please check the pdf file for more details.*

In this exercise you will:
    
- implement **Ridge Regression** to control overfitting
- implement **Logistic Regression with regularization** to control overfitting 
- implement **Cross-Validation** to control overfitting

In [2]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
digit_train = sio.loadmat('digit_train')
X = digit_train['X']
y = digit_train['y']
digit_test = sio.loadmat('digit_test')
X_test = digit_test['X_test']
y_test = digit_test['y_test']

In [3]:
def show_digit(fea):
    plt.rcParams['figure.figsize'] = (10.0, 8.0)
    idx = np.random.permutation(X.shape[1])
    fea = fea[:, idx[:100]]
    faceW = 28
    faceH = 28
    numPerLine = 20
    ShowLine = 4
    Y = np.zeros((faceH * ShowLine, faceW * numPerLine), dtype=np.float)
    for i in range(ShowLine):
        for j in range(numPerLine):
            Y[i * faceH:(i + 1) * faceH, j * faceW:(j + 1) * faceW] = fea[:,i * numPerLine + j].reshape((faceH, faceW))            
    plt.imshow(Y, cmap='gray')

### Ridge Regression and LOOCV

In [3]:
# Do feature normalization here
digit_train = sio.loadmat('digit_train')
X = digit_train['X']
y = digit_train['y']
digit_test = sio.loadmat('digit_test')
X_test = digit_test['X_test']
y_test = digit_test['y_test']
# show_digit(X)
print(X.shape, y.shape)
mean, std = np.mean(X, axis=1), np.std(X, axis=1)
X = (X - np.expand_dims(mean, -1)) / np.expand_dims(std + 1e-9, -1)
mean, std = np.mean(X_test, axis=1), np.std(X_test, axis=1)
X_test = (X_test - np.expand_dims(mean, -1)) / np.expand_dims(std + 1e-9, -1)

# Do LOOCV
lmbdas = np.array([1e-3, 1e-2, 1e-1, 0, 1, 1e1, 1e2, 1e3])
lmbda = 0
E_val_min = float('inf')

from ridge import ridge

for i in range(len(lmbdas)):
    print(lmbdas[i])
    E_val = 0
    E_w = 0
    for j in range(X.shape[1]):
        X_ = np.hstack((X[:, :j], X[:, (j+1):]))  # take point j out of X
        y_ = np.hstack((y[:, :j], y[:, (j+1):]))
        w = ridge(X_, y_, lmbdas[i])
        pred = np.sign(w[1:].T @ X[:, j] + w[0])
        E_val += np.sum(pred != y[:, j])
        E_w += np.sum(w**2)
    # Update lmbda according validation error
    print(E_val)
    E_val /= X.shape[1]
    print(E_val, E_w)
    if E_val < E_val_min:
        lmbda = lmbdas[i]
        E_val_min = E_val
print(lmbda)
w = ridge(X, y, lmbda)
# Compute training error
pred_train = np.sign(w[1:].T @ X + w[0])
error_train = np.sum(pred_train != y) / X.shape[1]
# Compute test error
pred_test = np.sign(w[1:].T @ X_test + w[0])
error_test = np.sum(pred_test != y_test) / X_test.shape[1]

print(error_train, error_test)

lmbda = 0
print(lmbda)
w = ridge(X, y, lmbda)
# Compute training error
pred_train = np.sign(w[1:].T @ X + w[0])
error_train = np.sum(pred_train != y) / X.shape[1]
# Compute test error
pred_test = np.sign(w[1:].T @ X_test + w[0])
error_test = np.sum(pred_test != y_test) / X_test.shape[1]

print(error_train, error_test)

(784, 200) (1, 200)
0.001
22
0.11 201.73593606801504
0.01
22
0.11 201.32779813909926
0.1
22
0.11 197.36519705247161
0.0
22
0.11 201.7814223755455
1.0
21
0.105 166.72766168917155
10.0
12
0.06 80.98224670214425
100.0
7
0.035 26.50951187392678
1000.0
7
0.035 6.737916094089704
100.0
0.0 0.05976896032144651
0
0.0 0.12606730286288298


### Logistic Regression with Regularization
Use the simlimar skeleton code above to implement it.

In [25]:
from logistic_r import logistic_r, logistic_function

digit_train = sio.loadmat('digit_train')
X = digit_train['X']
y = digit_train['y']
digit_test = sio.loadmat('digit_test')
X_test = digit_test['X_test']
y_test = digit_test['y_test']
print(X.shape, y.shape)
mean, std = np.mean(X, axis=1), np.std(X, axis=1)
X = (X - np.expand_dims(mean, -1)) / np.expand_dims(std + 1e-9, -1)
mean, std = np.mean(X_test, axis=1), np.std(X_test, axis=1)
X_test = (X_test - np.expand_dims(mean, -1)) / np.expand_dims(std + 1e-9, -1)

lmbdas = np.array([0, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3])
lmbda = 0
E_val_min = float('inf')
for i in range(len(lmbdas)):
    E_val = 0
    for j in range(X.shape[1]):
        X_ = np.hstack((X[:, :j], X[:, (j+1):]))  # take point j out of X
        y_ = np.hstack((y[:, :j], y[:, (j+1):]))
        y_[y_==-1] = 0
        w = logistic_r(X_, y_, lmbdas[i])
        y_[y_==0] = -1
        pred = np.sign(w[1:].T @ X[:, j] + w[0])
        E_val += np.sum(pred != y[:, j])
        
    # Update lmbda according validation error
    print(E_val)
    E_val /= X.shape[1]
    if E_val <= E_val_min:
        lmbda = lmbdas[i]
        E_val_min = E_val

def get_error(lmbda, X_train, y_train, X_test, y_test):
    y_train[y_train==-1] = 0
    w = logistic_r(X_train, y_train, lmbda)
    y_train[y_train==0] = -1
    pred_train = 1. / (np.exp(-(w[1:].T @ X_train + w[0])) + 1.)
    pred_train[pred_train >= 0.5] = 1
    pred_train[pred_train < 0.5] = -1
    error_train = np.sum(pred_train != y_train) / X_train.shape[1]
    pred_test = 1. / (np.exp(-(w[1:].T @ X_test + w[0])) + 1.)
    pred_test[pred_test >= 0.5] = 1
    pred_test[pred_test < 0.5] = -1
    error_test = np.sum(pred_test != y_test) / X_test.shape[1]
    return error_train, error_test

print(lmbda)

error_train, error_test = get_error(0, X, y, X_test, y_test)
print(error_train, error_test)
error_train, error_test = get_error(lmbda, X, y, X_test, y_test)
print(error_train, error_test)

(784, 200) (1, 200)
10
10
10
10
5
55
150
159
1.0
0.0 0.065796082370668
0.0 0.050226017076845805
